In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

### Bigram Model

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [3]:
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [7]:
# gradient descent
for k in range(1):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  #loss = -probs[torch.arange(num), ys].log().mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

3.7686190605163574


In [8]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  out = []
  ix = 0
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
      
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

texzmkloglquszipczktxhkmpmzistttwinmlgdukzka.
zr.
rocxtpucjwtsc.
gmtokmxczisqytxugkwpt.
dajkkluydjmscdgu.


### E01 Trigram Language Model

Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [9]:
chs = ['.']*2 + list("emma") + ['.']

In [10]:
for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    print(f'{ch1=}, {ch2=}, {ch3=}')

ch1='.', ch2='.', ch3='e'
ch1='.', ch2='e', ch3='m'
ch1='e', ch2='m', ch3='m'
ch1='m', ch2='m', ch3='a'
ch1='m', ch2='a', ch3='.'


In [11]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.']*2 + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = [stoi[ch1], stoi[ch2]]
    ix2 = stoi[ch3]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True) # X will be N x (27 * 2) = N x 54, so we need 54 neurons, giving 27 outputs.

number of examples:  228146


In [18]:
# gradient descent
for k in range(1):
  
  # forward pass
  x1 = F.one_hot(xs[:,0], 27).float()    # (N,27)
  x2 = F.one_hot(xs[:,1], 27).float()    # (N,27)
  xenc = torch.cat([x1, x2], dim=1)      # (N,54)
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -1 * W.grad

2.3549153804779053


In [19]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  out = []
  ix = [0, 0]
  while True:
    x1 = F.one_hot(torch.tensor(ix[0]), 27).float()    # (1,27)
    x2 = F.one_hot(torch.tensor(ix[1]), 27).float()    # (1,27)
    xenc = torch.cat([x1, x2]).reshape(1, 54)    # (1,54)
    # xenc = F.one_hot(torch.tensor(ix.reshape((-1, 54))), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
      
    ix = [ix[1], torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()]
    out.append(itos[ix[1]])
    if ix[1] == 0:
      break
  print(''.join(out))

cexze.
morlyurailaziaydamellimittain.
lusan.
ka.
da.


### E02 Evaluation

Split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

#### Bigram

In [36]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

train_xs, train_ys = xs[:int(0.8 * num)], ys[:int(0.8 * num)]
dev_xs, dev_ys = xs[int(0.8 * num):int(0.9 * num)], ys[int(0.8 * num):int(0.9 * num)]
test_xs, test_ys = xs[int(0.9 * num):], ys[int(0.9 * num):]

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [42]:
# gradient descent
for k in range(1):
  
  # forward pass
  xenc = F.one_hot(train_xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(train_xs.nelement()), train_ys].log().mean() + 0.01*(W**2).mean()
  #loss = -probs[torch.arange(num), ys].log().mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

2.452766180038452


In [49]:
def bigram_calc_loss(xs, ys, W):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(xs.nelement()), ys].log().mean()
    return loss

In [50]:
print(f"Bigram Model Dev loss: {bigram_calc_loss(dev_xs, dev_ys, W).item()}")
print(f"Bigram Model Test loss: {bigram_calc_loss(test_xs, test_ys, W).item()}")

Bigram Model Dev loss: 2.5989654064178467
Bigram Model Test loss: 2.602977752685547


#### Trigram

In [51]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.']*2 + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = [stoi[ch1], stoi[ch2]]
    ix2 = stoi[ch3]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

train_xs, train_ys = xs[:int(0.8 * num)], ys[:int(0.8 * num)]
dev_xs, dev_ys = xs[int(0.8 * num):int(0.9 * num)], ys[int(0.8 * num):int(0.9 * num)]
test_xs, test_ys = xs[int(0.9 * num):], ys[int(0.9 * num):]

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True) # X will be N x (27 * 2) = N x 54, so we need 54 neurons, giving 27 outputs.

number of examples:  228146


In [55]:
# gradient descent
for k in range(1):
  
  # forward pass
  x1 = F.one_hot(train_xs[:,0], 27).float()    # (N,27)
  x2 = F.one_hot(train_xs[:,1], 27).float()    # (N,27)
  xenc = torch.cat([x1, x2], dim=1)      # (N,54)
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(train_xs.shape[0]), train_ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -1 * W.grad

2.3210928440093994


In [59]:
def trigram_calc_loss(xs, ys, W):
    x1 = F.one_hot(xs[:,0], 27).float()
    x2 = F.one_hot(xs[:,1], 27).float()
    xenc = torch.cat([x1, x2], dim=1)
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(xs.shape[0]), ys].log().mean()
    return loss

In [61]:
print(f"Trigram Model Dev loss: {trigram_calc_loss(dev_xs, dev_ys, W).item()}")
print(f"Trigram Model Test loss: {trigram_calc_loss(test_xs, test_ys, W).item()}")

Trigram Model Dev loss: 2.5064163208007812
Trigram Model Test loss: 2.515704870223999


### E03 Tune Regularisation

Use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [63]:
# gradient descent
Ws = []
regs = [10, 1, 0.1, 0.01, 0.001, 0]
dev_losses = []
for reg in regs:
    W = torch.randn((54, 27), generator=g, requires_grad=True)
    for k in range(500):
      # forward pass
      x1 = F.one_hot(train_xs[:,0], 27).float()    # (N,27)
      x2 = F.one_hot(train_xs[:,1], 27).float()    # (N,27)
      xenc = torch.cat([x1, x2], dim=1)      # (N,54)
      logits = xenc @ W # predict log-counts
      counts = logits.exp() # counts, equivalent to N
      probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
      loss = -probs[torch.arange(train_xs.shape[0]), train_ys].log().mean() + reg*(W**2).mean()
      # print(loss.item())
      
      # backward pass
      W.grad = None # set to zero the gradient
      loss.backward()
      
      # update
      W.data += -5 * W.grad
    
    dev_loss = trigram_calc_loss(dev_xs, dev_ys, W).item()
    print(f"Dev Loss: {dev_loss}")
    dev_losses.append(dev_loss)
    
    Ws.append(W)

Dev Loss: 2.921194314956665
Dev Loss: 2.6151700019836426
Dev Loss: 2.5602316856384277
Dev Loss: 2.548191785812378
Dev Loss: 2.5527312755584717
Dev Loss: 2.559356212615967


In [76]:
dev_losses # U curve

[2.921194314956665,
 2.6151700019836426,
 2.5602316856384277,
 2.548191785812378,
 2.5527312755584717,
 2.559356212615967]

In [75]:
best_dev_loss = min(dev_losses)
best_dev_index = np.argmin(dev_losses).item()
best_reg = regs[best_dev_index]
print(f"{best_dev_loss=} at index {best_dev_index}, {best_reg=}")

best_dev_loss=2.548191785812378 at index 3, best_reg=0.1


In [77]:
trigram_calc_loss(test_xs, test_ys, Ws[best_dev_index]).item()

2.5554730892181396

### E04 One-hot selection.

We saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [78]:
def trigram_calc_loss(xs, ys, W):
    x1 = F.one_hot(xs[:,0], 27).float()
    x2 = F.one_hot(xs[:,1], 27).float()
    xenc = torch.cat([x1, x2], dim=1)
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(xs.shape[0]), ys].log().mean()
    return loss

In [126]:
def trigram_calc_loss_opt(xs, ys, W):
    logits = W[xs[:, 0]] + W[27 + xs[:, 1]] # indexing
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(xs.shape[0]), ys].log().mean()
    return loss

In [127]:
trigram_calc_loss(test_xs, test_ys, W)

tensor(2.5643, grad_fn=<NegBackward0>)

In [128]:
trigram_calc_loss_opt(test_xs, test_ys, W)

tensor(2.5643, grad_fn=<NegBackward0>)

### E05 Cross Entropy

Look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [131]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True) # X will be N x (27 * 2) = N x 54, so we need 54 neurons, giving 27 outputs.

In [147]:
# gradient descent
for k in range(1):
  
  # forward pass
  x1 = F.one_hot(train_xs[:,0], 27).float()    # (N,27)
  x2 = F.one_hot(train_xs[:,1], 27).float()    # (N,27)
  xenc = torch.cat([x1, x2], dim=1)      # (N,54)
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  # loss = -probs[torch.arange(train_xs.shape[0]), train_ys].log().mean() + 0.01*(W**2).mean()
  loss = F.cross_entropy(logits, train_ys) + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -1 * W.grad

2.334840774536133


Therefore, it is the same as the loss we calculated before. Also more stable and better optimised.